<a href="https://colab.research.google.com/github/mhanauer/xgboost_drop_out_colab/blob/master/data_xgboost_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load in the repostiory through git.

First git clone the porject

In [2]:
!git clone https://github.com/mhanauer/xgboost_colab_example.git

Cloning into 'xgboost_colab_example'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [75]:
!pip install xgboost
!pip install lightgbm
!pip install scikit-optimize
import pandas as pd
import sklearn
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import numpy as np
from joblib import dump, load
from sklearn.model_selection import RepeatedKFold
%cd xgboost_colab_example/

[Errno 2] No such file or directory: 'xgboost_colab_example/'
/content/xgboost_colab_example


Now we review the data

In [4]:
housing_data_set = pd.read_csv("housing_data_set.csv")
housing_data_set.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


Check the types

In [5]:
housing_data_set.dtypes

id                 int64
date              object
price              int64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

Steps: Feature engineering
Pipe function for dropping variables and change date

In [41]:
y = housing_data_set.price

def date_quarter_clean(x):
  x['date'] = pd.to_datetime(x['date'])
  x["year_quarter"] = x['date'].dt.to_period('Q')
  return x

def dummies_fun(x):
  x_dummies = pd.get_dummies(x.year_quarter, prefix = "year_quarter")
  x = pd.concat([x, x_dummies], axis = 1)
  return x

def filter_fun(x): 
  x = x.drop(columns = ['price', 'id', 'date', 'year_quarter']) 
  return x

x_clean = (housing_data_set.pipe(date_quarter_clean)
                  .pipe(dummies_fun)
                  .pipe(filter_fun))
x_clean

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,zipcode,lat,long,sqft_living15,sqft_lot15,year_quarter_2014Q2,year_quarter_2014Q3,year_quarter_2014Q4,year_quarter_2015Q1,year_quarter_2015Q2
0,3,1.00,1180,5650,1.0,0,0,3,7,1180,...,98178,47.5112,-122.257,1340,5650,0,0,1,0,0
1,3,2.25,2570,7242,2.0,0,0,3,7,2170,...,98125,47.7210,-122.319,1690,7639,0,0,1,0,0
2,2,1.00,770,10000,1.0,0,0,3,6,770,...,98028,47.7379,-122.233,2720,8062,0,0,0,1,0
3,4,3.00,1960,5000,1.0,0,0,5,7,1050,...,98136,47.5208,-122.393,1360,5000,0,0,1,0,0
4,3,2.00,1680,8080,1.0,0,0,3,8,1680,...,98074,47.6168,-122.045,1800,7503,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,3,2.50,1530,1131,3.0,0,0,3,8,1530,...,98103,47.6993,-122.346,1530,1509,1,0,0,0,0
21609,4,2.50,2310,5813,2.0,0,0,3,8,2310,...,98146,47.5107,-122.362,1830,7200,0,0,0,1,0
21610,2,0.75,1020,1350,2.0,0,0,3,7,1020,...,98144,47.5944,-122.299,1020,2007,1,0,0,0,0
21611,3,2.50,1600,2388,2.0,0,0,3,8,1600,...,98027,47.5345,-122.069,1410,1287,0,0,0,1,0


Step: Conduct inital testing training split

In [42]:
x_train, x_test, y_train, y_test = train_test_split(
    x_clean, y, test_size=0.30, random_state=42)

Step: Set up the tuner

rmsle: Can overcome errors with outliers as long as no negative values.

tree_method: Auto selects the method that is best generally based on the data size (e.g., exact for small and approx for large).

learning_rate: Learning rate how fast the algorithm learns.  Between zero and one.

max_depth: Max depth of cuts in tree.  Default is six and documentation gives warning about not going to high.

subsample: Percentage of training data to start the model

colsample_bytree: Percentage of features selected

reg_lambda: Defaults to one which means no impact and lower values impact the effect of predictors to low by but none zero.  For example, if age a had a small impact on the outcome the impact would be shrunk to zero.



In [66]:

estimator = xgb.XGBRegressor(
  objective = 'reg:squarederror', 
  eval_metric = 'rmse', 
  tree_metod = 'auto')
 
search_spaces = {
  'learning_rate': (0.001, 1.0),
  'max_depth': (0, 20),
  'subsample': (0.1, 1.0),
  'colsample_bytree': (0.01, 1.0),
  'reg_lambda': (1e-9, 1.0)
    }

Step: set up the cross validation

In [76]:
cv = RepeatedKFold(n_splits=2, n_repeats=2, random_state=2652124)
n_interations = 50

Step: Set up the bayes search

In [83]:
bayes_cv_tuner = BayesSearchCV(
  estimator = estimator, 
  search_spaces = search_spaces,
  cv = cv, 
  verbose = 0, 
  refit = True
)

Step: Run the model

In [84]:
result_bayes_tune = bayes_cv_tuner.fit(x_train, y_train)
result_bayes_tune

BayesSearchCV(cv=RepeatedKFold(n_repeats=2, n_splits=2, random_state=2652124),
              estimator=XGBRegressor(eval_metric='rmse',
                                     objective='reg:squarederror',
                                     tree_metod='auto'),
              search_spaces={'colsample_bytree': (0.01, 1.0),
                             'learning_rate': (0.001, 1.0),
                             'max_depth': (0, 20), 'reg_lambda': (1e-09, 1.0),
                             'subsample': (0.1, 1.0)})

Step: Build a function to use the final model to get the final statistics

In [90]:
best_estimator_bayes_tune = result_bayes_tune.best_estimator_
def run_experiment(model):
    model.fit(x_test, y_test)
    y_pred = model.predict(x_test)
    print("R^2 : ", r2_score(y_test, y_pred))
    print("MAE :", mean_absolute_error(y_test,y_pred))
    print("RMSE:",np.sqrt(mean_squared_error(y_test, y_pred)))


Step: Run the function and evaluate the model

In [91]:
results_xgboost_housing = run_experiment(best_estimator_bayes_tune)
results_xgboost_housing

R^2 :  0.9872060109847683
MAE : 31362.513499575878
RMSE: 42977.0153726326


Step: Pickle the model

In [94]:
best_estimator_bayes_tune = load('best_estimator_bayes_tune.joblib') 
best_estimator_bayes_tune

XGBRegressor(colsample_bytree=1.0, eval_metric='rmse',
             learning_rate=0.2316312292448271, max_depth=6, missing=nan,
             objective='reg:squarederror', reg_lambda=0.6825497578292735,
             subsample=1.0, tree_metod='auto')

Step: Add git pickle model

In [95]:
!git add best_estimator_bayes_tune.joblib